# Data Extraction, Cleaning, Transformation, and Loading of Olympics History Data

<img src="https://media.licdn.com/dms/image/D4D03AQEnxU1bGqE73Q/profile-displayphoto-shrink_400_400/0/1678754238733?e=1722470400&v=beta&t=Kw7qF2f0hppi8H-VNTFkrWpXl0R1odG2lUuQc0e6kd4" alt="Profile" width="100" height="45" style="float: left; margin-right: 10px;">

## By: Carlos Varela

## Introduction
In this notebook, we will focus on the essential steps of data cleaning and transformation, which are critical to preparing our dataset for analysis and modeling. Data cleaning involves detecting and correcting (or removing) errors and inconsistencies in data to improve its quality. Data transformation, on the other hand, involves transforming raw data into a format suitable for analysis, which often includes restructuring, enriching, and merging data from various sources. Finally, we will write a dedicated pipeline to loading the transformed data into a new PostgreSQL databse. 

## Purpose of Data Cleaning & Transformation

Data cleaning and transformation are foundational processes in the data analysis pipeline. These steps ensure that the data is accurate, complete, and structured appropriately for subsequent analysis and modeling. The main objectives of data cleaning and transforming in this project are:

- **Data Quality Improvement:** Identifying and rectifying errors, inconsistencies, and missing values in the dataset to ensure data integrity. This step is crucial to prevent any inaccurate or misleading results in the analysis.
- **Standardization and Normalization:** Ensuring that the data is in a consistent format. This includes standardizing date formats, units of measure, and categorical values to facilitate accurate comparisons and aggregations.
- **Data Enrichment:** Integrating additional relevant information into the dataset, which might involve merging data from different sources or deriving new features from existing data to enhance the dataset's richness and utility.
- **Handling Missing Values:** Employing strategies such as imputation, interpolation, or removal of missing values to deal with incomplete data, thereby maintaining the dataset's usability.

Overall, we aim to prepare a high-quality, reliable dataset that will serve as a robust foundation for subsequent data analysis using SQL, and Tableau for visualization.

## Data Extraction

For this project I will be using the dataset "120 years of Olympic history: athletes and results" collected and wrangled by RGRIFFIN and found on Kaggle: https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results?resource=download

The dataset contains historical data on the modern Olympic Games from Athens 1896 and until Rio 2016. The author of the dataset reported scrapping this data from www.sports-reference.com in May of 2018.

You can learn more about this dataset and the author here: https://www.kaggle.com/heesoo37

In [1]:
# Importing the necessary libraries:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Retreiving current working directory and adding folder path:
path = os.path.join('raw_data')

# Opening dataset as pandas df for inspection of the head and tail:
raw_athlete_data = pd.read_csv(path+'/athlete_events.csv')
raw_athlete_data.head(5)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [3]:
raw_athlete_data.tail(5)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN
271115,135571,Tomasz Ireneusz ya,M,34.0,185.0,96.0,Poland,POL,2002 Winter,2002,Winter,Salt Lake City,Bobsleigh,Bobsleigh Men's Four,NaN


In [4]:
raw_athlete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


## Data Cleaning

It looks like our data might require some extra wrangling before we begin our analysis. We can already build an initial assessment of this dataset by just inspecting the head, the tail, and the dataframe information.
The data is comprised of 13 columns. Every record describes athletes' information based on the year of the olympics session. We seem to have some duplicates as well as null values. There appear to be 271116 records in the dataset, however, after removing duplicates and inspecting quality, we might end up with less. 

Now we will: 
- Inspect for duplicated records and remove them
- Lowercasing every string value
- Deleting leading or trailing spaces that might impact data manipulation tasks
- Inspect for null values and handle them

In [5]:
preprocessed_athletes_data = raw_athlete_data.copy() # In case we need the original df...

# Lowercasing all strings for easier manipulation:
preprocessed_athletes_data = preprocessed_athletes_data.applymap(lambda x: x.lower() if type(x) == str else x)
preprocessed_athletes_data.columns = map(str.lower, preprocessed_athletes_data.columns)

# deleting all leading and trailing white spaces:
preprocessed_athletes_data = preprocessed_athletes_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#inspecting head:
preprocessed_athletes_data.head(1)

,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
0,1,a dijiang,m,24.0,180.0,80.0,china,chn,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,NaN


## Handling duplicates

For the scope of this analysis we will remove all duplicates.

In [6]:
# Removing duplicates:
if preprocessed_athletes_data.duplicated().sum() > 0:
    preprocessed_athletes_data.drop_duplicates()
    print('Duplicated records deleted')
else:
    print('No duplicates found')

Duplicated records deleted


## Handling Null Values

Our dataset comes with null values for different records. Depending on the attribute, we might want to handle them differently. First of all let's review which attributes contain null values.

In [7]:
print('Columns with nulls')
nulls_data = preprocessed_athletes_data.isna().sum()
nulls_data = nulls_data[nulls_data > 0]
nulls_data

Columns with nulls


age         9474
height     60171
weight     62875
medal     231333
dtype: int64

Since deleting columns with null values might negatively impact our ability to analyze this dataset, we will handle null values from each column in a different way. 

- age: Two options to deal with null values depending on how many athletes have missing values
    - Option a: Augment the dataset by looking for the Athlete's age
    - Option b: Use the mean age of the category in which they are competing for that particular year
- height: Two options to deal with null values depending on how many atheltes miss this piece
    - Option a: Augment the dataset by looking for the Athlete's age
    - Option b: Use the mean height of the category in which they are competing for that particular year
- weight: Two options to deal with null values depending on how many atheltes miss this piece
    - Option a: Augment the dataset by looking for the Athlete's age
    - Option b: Use the mean weight of the category in which they are competing for that particular year
- Medal: We will create the category "no medal" for all athletes who did not win a medal.

In [8]:
# starting with Medal since it is the simpler approach:
preprocessed_athletes_data['medal'] = preprocessed_athletes_data['medal'].fillna('no_medal')
preprocessed_athletes_data['medal'].unique()

array(['no_medal', 'gold', 'bronze', 'silver'], dtype=object)

In [9]:
# Getting the number of unique athletes:
preprocessed_athletes_data['name'].nunique()

134730

In [10]:
# Isolating all records from athletes without age information:
no_age_athletes_data = preprocessed_athletes_data[preprocessed_athletes_data['age'].isna()]
print('Number of unique athletes with no Age data: ',no_age_athletes_data['name'].nunique())

# Obtaining percentage of athletes with no age data:
percentage = (no_age_athletes_data['name'].nunique()/preprocessed_athletes_data['name'].nunique())*100
print(f'{percentage:.2f}% of athletes contain no age data')

Number of unique athletes with no Age data:  6356
4.72% of athletes contain no age data


Since we wish to conduct exploratory data analysis on trends related to age, height and weight, we need to account for null values in a way that does not involve deleting those records. My approach will be to augment the dataset using mean values. In the following segment we will compute the statistical mean of age, height, and weight for each sport during each edition of the olympics. Then, we will assign the mean values obtained to each athletes missing this information only when it matches their discipline and year of participation.

In [11]:
# Isolate records per year & keeping age, height, weight, year, and sport:
columns_to_keep = ['sex','age', 'height', 'weight', 'year', 'sport']
average_stats_by_sport = preprocessed_athletes_data[columns_to_keep]

# Grouping by year & discipline while averaging age, height, and weight using mean:
average_stats_by_sport = average_stats_by_sport.groupby(['sex','year','sport'],as_index=False)[['age',
                                                                                                'height',
                                                                                                'weight']].mean()

In [12]:
# Rounding age to no decimal points while keeping 1 decimal for height and weight:
average_stats_by_sport = average_stats_by_sport.round({'age':0, 'height':1, 'weight':1})

# Visualizing the resulting df for reference:
average_stats_by_sport

,sex,year,sport,age,height,weight
0,f,1900,croquet,40.0,NaN,NaN
1,f,1900,equestrianism,NaN,NaN,NaN
2,f,1900,golf,31.0,NaN,NaN
3,f,1900,sailing,32.0,NaN,NaN
4,f,1900,tennis,24.0,NaN,NaN
...,...,...,...,...,...,...
1391,m,2016,triathlon,27.0,178.7,66.9
1392,m,2016,volleyball,28.0,196.9,89.6
1393,m,2016,water polo,28.0,191.4,94.7
1394,m,2016,weightlifting,25.0,170.8,87.4


### Filling in Null values

Now I will merge the average statistics with the original dataframe to align each athlete's record with the corresponding average statistics based on the year and sport. This is a step necessary to replace null values with their corresponding average values.

In [13]:
# Merge with original dataframe on sex, year, and sport:
merged_preprocessed_athletes_data = pd.merge(
    preprocessed_athletes_data, #original df...
    average_stats_by_sport, # df to merge with...
    on=['sex', 'year', 'sport'], # using these columns as keys to join omn
    suffixes=('', '_average') # assigning new names to the merged columns
)

# Visualizing result:
merged_preprocessed_athletes_data.head(5)

,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal,age_average,height_average,weight_average
0,1,a dijiang,m,24.0,180.0,80.0,china,chn,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal,27.0,199.5,97.3
1,2256,vladan alanovi,m,25.0,190.0,81.0,croatia,cro,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,silver,27.0,199.5,97.3
2,2424,santiago aldama aleson,m,23.0,213.0,98.0,spain,esp,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal,27.0,199.5,97.3
3,4120,armin andres,m,33.0,180.0,76.0,germany,ger,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal,27.0,199.5,97.3
4,4140,enrique andreu balbuena,m,24.0,210.0,102.0,spain,esp,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal,27.0,199.5,97.3


Now that we have merged the average statistics with the original dataframe I will replace null values with their corresponding averages at a record level.

In [14]:
# Fill null values with the corresponding average values:
merged_preprocessed_athletes_data['age'] = merged_preprocessed_athletes_data['age'].fillna(
    merged_preprocessed_athletes_data['age_average'])
merged_preprocessed_athletes_data['height'] = merged_preprocessed_athletes_data['height'].fillna(
    merged_preprocessed_athletes_data['height_average'])
merged_preprocessed_athletes_data['weight'] = merged_preprocessed_athletes_data['weight'].fillna(
    merged_preprocessed_athletes_data['weight_average'])

# Drop the average columns since we don't need them anymore:
final_athletes_data = merged_preprocessed_athletes_data.drop(columns=['age_average', 
                                                                      'height_average', 
                                                                      'weight_average'])

# Visualizing final df for inspection:
final_athletes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      271116 non-null  int64  
 1   name    271116 non-null  object 
 2   sex     271116 non-null  object 
 3   age     271113 non-null  float64
 4   height  268741 non-null  float64
 5   weight  264732 non-null  float64
 6   team    271116 non-null  object 
 7   noc     271116 non-null  object 
 8   games   271116 non-null  object 
 9   year    271116 non-null  int64  
 10  season  271116 non-null  object 
 11  city    271116 non-null  object 
 12  sport   271116 non-null  object 
 13  event   271116 non-null  object 
 14  medal   271116 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


Pretty close now! Since some of the remaining null values are linked to won medals we can't simply eliminate them. I will deal with the remaining null values by using the historical average of each sport for both men and women.

In [15]:
# Applying the same pipeline as above

# Group by sex and sport:
grouped = final_athletes_data.groupby(['sex', 'sport'])

# Create columns for the mean values but now using transform:
final_athletes_data['age_mean'] = grouped['age'].transform('mean')
final_athletes_data['height_mean'] = grouped['height'].transform('mean')
final_athletes_data['weight_mean'] = grouped['weight'].transform('mean')

# Fill the missing values with the new mean values:
final_athletes_data['age'] = final_athletes_data['age'].fillna(final_athletes_data['age_mean'])
final_athletes_data['height'] = final_athletes_data['height'].fillna(final_athletes_data['height_mean'])
final_athletes_data['weight'] = final_athletes_data['weight'].fillna(final_athletes_data['weight_mean'])

# Drop the temporary mean columns as we don't need them anymore:
final_athletes_data = final_athletes_data.drop(columns=['age_mean', 'height_mean', 'weight_mean'])

# Inspecting result:
print(final_athletes_data.info())
print()
final_athletes_data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      271116 non-null  int64  
 1   name    271116 non-null  object 
 2   sex     271116 non-null  object 
 3   age     271116 non-null  float64
 4   height  271016 non-null  float64
 5   weight  270521 non-null  float64
 6   team    271116 non-null  object 
 7   noc     271116 non-null  object 
 8   games   271116 non-null  object 
 9   year    271116 non-null  int64  
 10  season  271116 non-null  object 
 11  city    271116 non-null  object 
 12  sport   271116 non-null  object 
 13  event   271116 non-null  object 
 14  medal   271116 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB
None



,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
0,1,a dijiang,m,24.0,180.0,80.0,china,chn,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal
1,2256,vladan alanovi,m,25.0,190.0,81.0,croatia,cro,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,silver
2,2424,santiago aldama aleson,m,23.0,213.0,98.0,spain,esp,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal
3,4120,armin andres,m,33.0,180.0,76.0,germany,ger,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal
4,4140,enrique andreu balbuena,m,24.0,210.0,102.0,spain,esp,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal


Uppon further investigation, I have determined that the missing values are related to competitors from the following sports:
- art competitions
- polo
- cricket
- racquets
- military ski patrol
- croquet
- jeu de paume
- roque
- alpinism
- basque pelota
- motorboating
- aeronautics 

Some of these might not appear in recent editions or have enough competitors to come up with a statistically significant metric. Therefore, we will use the nationality and sex to come up with mean values for the remaining competitors.

In [16]:
# Applying the same pipeline as above

# Group by sex and NOC to account for teams with different names to their countries:
grouped_by_country = final_athletes_data.groupby(['sex', 'noc'])

# Create columns for the mean values but now using transform:
final_athletes_data['height_mean'] = grouped_by_country['height'].transform('mean')
final_athletes_data['weight_mean'] = grouped_by_country['weight'].transform('mean')

# Fill the missing values with the new mean values:
final_athletes_data['height'] = final_athletes_data['height'].fillna(final_athletes_data['height_mean'])
final_athletes_data['weight'] = final_athletes_data['weight'].fillna(final_athletes_data['weight_mean'])

# Drop the temporary mean columns as we don't need them anymore:
final_athletes_data = final_athletes_data.drop(columns=['height_mean', 'weight_mean'])

# Inspecting result:
print(final_athletes_data.info())
print()
final_athletes_data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      271116 non-null  int64  
 1   name    271116 non-null  object 
 2   sex     271116 non-null  object 
 3   age     271116 non-null  float64
 4   height  271116 non-null  float64
 5   weight  271116 non-null  float64
 6   team    271116 non-null  object 
 7   noc     271116 non-null  object 
 8   games   271116 non-null  object 
 9   year    271116 non-null  int64  
 10  season  271116 non-null  object 
 11  city    271116 non-null  object 
 12  sport   271116 non-null  object 
 13  event   271116 non-null  object 
 14  medal   271116 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB
None



,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
0,1,a dijiang,m,24.0,180.0,80.0,china,chn,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal
1,2256,vladan alanovi,m,25.0,190.0,81.0,croatia,cro,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,silver
2,2424,santiago aldama aleson,m,23.0,213.0,98.0,spain,esp,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal
3,4120,armin andres,m,33.0,180.0,76.0,germany,ger,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal
4,4140,enrique andreu balbuena,m,24.0,210.0,102.0,spain,esp,1992 summer,1992,summer,barcelona,basketball,basketball men's basketball,no_medal


In [17]:
# Transforming age to int for future manipulation:
final_athletes_data['age'] = final_athletes_data['age'].astype(int)

For better data manipulation and analysis we will now sort the records in the following order:
- Year: Ascending order for time series analysis
- Sport, Event, Sex, Team, NOC, and Name in alphabetical order

In [18]:
# Sort listed values, by default sort_values takes ascending true:
final_athletes_data_sorted = final_athletes_data.sort_values(by = ['year',
                                      'sex',
                                      'sport',
                                      'event',
                                      'team',
                                      'noc',
                                      'name'])

# Inspecting final result:
final_athletes_data_sorted.head(10)

,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
245062,35698,"edwin harold ""teddy"" flack",m,22,174.9,70.1,australia,aus,1896 summer,1896,summer,athina,athletics,"athletics men's 1,500 metres",gold
245102,68911,albin georges lermusiaux,m,21,174.9,70.1,france,fra,1896 summer,1896,summer,athina,athletics,"athletics men's 1,500 metres",bronze
245064,38123,carl galle,m,23,154.0,45.0,germany,ger,1896 summer,1896,summer,athina,athletics,"athletics men's 1,500 metres",no_medal
245060,35094,angelos fetsis,m,22,174.9,70.1,greece,gre,1896 summer,1896,summer,athina,athletics,"athletics men's 1,500 metres",no_medal
245132,120917,"dimitrios mikhail ""dimitri"" tombrof",m,18,174.9,70.1,greece,gre,1896 summer,1896,summer,athina,athletics,"athletics men's 1,500 metres",no_medal
245074,41160,dimitrios p. golemis,m,21,174.9,70.1,greece,gre,1896 summer,1896,summer,athina,athletics,"athletics men's 1,500 metres",no_medal
245093,57441,konstantinos karakatsanis,m,22,174.9,70.1,greece,gre,1896 summer,1896,summer,athina,athletics,"athletics men's 1,500 metres",no_medal
245039,12068,arthur charles blake,m,24,174.9,70.1,united states,usa,1896 summer,1896,summer,athina,athletics,"athletics men's 1,500 metres",silver
245117,107090,eugen stahl schmidt,m,34,174.9,70.1,denmark,den,1896 summer,1896,summer,athina,athletics,athletics men's 100 metres,no_medal
245078,43020,alphonse grisel,m,22,174.9,70.1,france,fra,1896 summer,1896,summer,athina,athletics,athletics men's 100 metres,no_medal


Finally, we have a dataset ready for analysis, with no duplicated or null values. For the scope of this analysis we will use SQL. Therefore, our next step will be to load this content into a database for data modeling.

## Data Loading

In [19]:
# For anyone replicating this code: Make sure you have the proper JDBC drivers to connect to Postgresql

# Import necessary libraries:
import psycopg2
from psycopg2 import sql # Using sql module instead of string concat to avoid sql injection...
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from dotenv import load_dotenv # to read env variables...

# Get credentials from environment variables:
load_dotenv('.env')

db_name = os.getenv('DBNAME')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
host = os.getenv('HOST')
port = os.getenv('PORT')

In [20]:
# Assigning default db:
default_db = 'spark_streaming'

# Declaring our connection:
conn = psycopg2.connect(dbname=default_db,
      user = user, 
      password = password,
      host = host,
      port = port)

# Employin autocommit:
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Creating a cursor:
curr = conn.cursor()

# Create a new database if not exists:
try:
      curr.execute(sql.SQL("CREATE DATABASE {}").format(
            sql.Identifier(db_name)))
      curr.close()
      conn.close()
      print(f'Database {db_name} created succesfully')

except Exception as e:
      print('Could not create database: ', e)

Could not create database, error:  database "olympics" already exists



In [21]:
# lets define a SQLAlchemy engine for operations management:
from sqlalchemy import create_engine

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}')

# Define our table name:
table_name = 'athletes_data'

# send the data:
final_athletes_data_sorted.to_sql(table_name, engine, if_exists='replace', index=False) #change to append TBD...
print(f'DataFrame stored in table "{table_name}" successfully.')

DataFrame stored in table "athletes_data" successfully.


## Scraping data for Cost of Hosting the Olympics

Our next step will be to extract the dataset related to the costs of hosting the olympics. The dataset we will be using is the table related to the article "Cost of the Olympic Games" found in: https://en.wikipedia.org/wiki/Cost_of_the_Olympic_Games. This table contains data collected from 70 different sources. As stated by the author in the article: *"The table contains both the operating costs and total final costs (which include various infrastructure upgrades and security costs), as well as both known and not estimated figures. Net loss or gain are measured against the operating budgets. Intangible costs (such as to the environment and society) and benefits (through tourism) are not included here."*

In [22]:
# Scraping all tables from source using pandas:
tables = pd.read_html('https://en.wikipedia.org/wiki/Cost_of_the_Olympic_Games')
print(f' Found {len(tables)} tables')

for i, table in enumerate(tables): print(f'table number {i}: ',tables[i])

 Found 3 tables
table number 0:      0                                                  1
0 NaN  This article possibly contains original resear...
table number 1:      0                                                  1
0 NaN  This section does not cite any sources. Please...
table number 2:                  Olympiad                         Host  \
0   1896 Summer Olympics                       Athens   
1   1900 Summer Olympics                        Paris   
2   1904 Summer Olympics                    St. Louis   
3   1908 Summer Olympics                       London   
4   1912 Summer Olympics                    Stockholm   
5   1920 Summer Olympics                      Antwerp   
6   1924 Winter Olympics                     Chamonix   
7   1924 Summer Olympics                        Paris   
8   1928 Winter Olympics                   St. Moritz   
9   1928 Summer Olympics                    Amsterdam   
10  1932 Winter Olympics                  Lake Placid   
11  1932 Summer Olymp

For the current analysis, we will only be using table 2 since it is the one containing the information of costs attributed to hosting the olympics. The next steps will be to inspect, clean, and wrangle the data just like with the previous dataset.

In [23]:
# Naming and inspecting our dataframe:
olympics_costs = tables[2].copy() # in case we need the original
olympics_costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Olympiad                59 non-null     object
 1   Host                    59 non-null     object
 2   Final Operating Budget  30 non-null     object
 3   Total Costs             22 non-null     object
 4   Taxpayer Contribution   15 non-null     object
 5   Profit/Loss             29 non-null     object
 6   Year Debt Paid Off      12 non-null     object
 7   Notes                   15 non-null     object
dtypes: object(8)
memory usage: 3.8+ KB


Let us describe each column in the dataset with the information provided in the data source:

- **Olympiad**: The year and season representing the edition of the Olympic Games.
- **Host**: The city where the Olympic Games were held.
- **Final Operating Budget**: The final budget allocated for the operational costs of hosting the Olympic Games.
- **Total Costs**: The overall expenditure including both operating costs and additional infrastructure and security expenses for hosting the Olympic Games.
- **Taxpayer Contribution**: The amount of public funding provided by taxpayers to support the costs of hosting the Olympic Games.
- **Profit/Loss**: The net financial gain or deficit resulting from the total operating budget subtracted by the total revenue generated.
- **Year Debt Paid Off**: The year by which any incurred debt from hosting the Olympic Games was fully repaid.
- **Notes**: Additional information or clarifications related to the financial figures or context of the Olympic Games.

### Observations

- There are missing values for 6 of the attributes
- The information provided for each record has been scraped from multiple sources
- All attributes are of type object, meaning we will need to perform cleaning to manipulate numbers and dates
- Since there are only 59 records in the set we might be able to manually augment it through research

In [24]:
print('Dataframe head:')
olympics_costs.head()

Dataframe head:


,Olympiad,Host,Final Operating Budget,Total Costs,Taxpayer Contribution,Profit/Loss,Year Debt Paid Off,Notes
0,1896 Summer Olympics,Athens,NaN,"3,740,000 ₯[6]",NaN,NaN,NaN,"Donations by George Averoff of 1,000,000 ₯ cov..."
1,1900 Summer Olympics,Paris,NaN,NaN,NaN,NaN,NaN,NaN
2,1904 Summer Olympics,St. Louis,NaN,NaN,NaN,NaN,NaN,NaN
3,1908 Summer Olympics,London,NaN,"US$394,000 est.[6]",NaN,"£6,377",NaN,NaN
4,1912 Summer Olympics,Stockholm,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
print('Dataframe tail: ')
olympics_costs.tail(6)

Dataframe tail: 


,Olympiad,Host,Final Operating Budget,Total Costs,Taxpayer Contribution,Profit/Loss,Year Debt Paid Off,Notes
53,2024 Summer Olympics,Paris,US$8.2 billion[68],NaN,US$3.22 billion[69],NaN,NaN,NaN
54,2026 Winter Olympics,Milan and Cortina d'Ampezzo,US$1.6 billion[70],NaN,NaN,NaN,NaN,NaN
55,2028 Summer Olympics,Los Angeles,NaN,NaN,NaN,NaN,NaN,NaN
56,2030 Winter Olympics,French Alps,NaN,NaN,NaN,NaN,NaN,NaN
57,2032 Summer Olympics,Brisbane,NaN,NaN,NaN,NaN,NaN,NaN
58,2034 Winter Olympics,Salt Lake City–Utah,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Declaring columns of interest:
columns_of_interest = ['Olympiad',
                       'Host',
                       'Final Operating Budget',
                       'Total Costs',
                       'Taxpayer Contribution',
                       'Profit/Loss']

# Filtering by columns of interest:
olympics_costs_no_nulls = olympics_costs[columns_of_interest].dropna()
olympics_costs_no_nulls

,Olympiad,Host,Final Operating Budget,Total Costs,Taxpayer Contribution,Profit/Loss
33,1984 Summer Olympics,Los Angeles,"US$320,000,000[13]","US$413,000,000[13]","$75,000,000[14][15]","US$250,000,000[citation needed]"
34,1988 Winter Olympics,Calgary,"CDN$438,000,000[16]","CDN$899,000,000[16]","CDN$425,000,000[16]","CDN$32,000,000[16]"
41,2000 Summer Olympics,Sydney,"A$6,600,000,000 [24][25]","A$3,000,000,000 (A$3,635,000,000 borne by the ...","A$2,050,000,000[26]",US$2.1 billion [27]
42,2002 Winter Olympics,Salt Lake City,"US$2,000,000,000 [28]","US$1,200,000,000 [29]","US$600,000,000[30]","US$101,000,000[31]"
43,2004 Summer Olympics,Athens,"US$15,000,000,000[32]","US$9,000,000,000[33]","US$6,200,000,000[34]","US$14,500,000[35]"
46,2010 Winter Olympics,Vancouver,"CDN$ 1,700,000,000 (US$1,260,000,000)[41]","US$ 6,400,000,000[42]","US$ 2,300,000,000[43]","CDN$ 1,900,000[44]"
47,2012 Summer Olympics,London,£GBP6583688684.12[48],"US$14,600,000,000[49]","US$4,400,000,000[50]",GBP £nil[51]


We will keep this as a separate dataframe for reference when augmenting the records that contain null values.

## Data Cleaning and Transformation

For this cleaning stage, I will perform the following pipeline:
- Lowercasing all string values to achieve a uniform format
- checking for and deleting duplicated records
- Creating a column to isolate the year of the edition
- Transforming all currencies to US Dollars adjusted to 2024 for inflation
- Removing special characters and text from numeric values when needed

In [27]:
# Lowercasing all values:
olympics_costs_processed = olympics_costs.applymap(lambda x: x.lower() if type(x)==str else x)
olympics_costs_processed.columns = map(str.lower, olympics_costs_processed.columns)

# Visualizing dataframe for inspection:
olympics_costs_processed.head(3)

,olympiad,host,final operating budget,total costs,taxpayer contribution,profit/loss,year debt paid off,notes
0,1896 summer olympics,athens,NaN,"3,740,000 ₯[6]",NaN,NaN,NaN,"donations by george averoff of 1,000,000 ₯ cov..."
1,1900 summer olympics,paris,NaN,NaN,NaN,NaN,NaN,NaN
2,1904 summer olympics,st. louis,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Deleting duplicates if found:


Insights and Analysis
Trend Analysis

Identify trends in the costs and taxpayer contributions over the decades. Are the costs increasing? Is the proportion of taxpayer contributions rising or falling?
Economic Impact

Analyze the relationship between taxpayer contributions and final profit/loss. Do higher taxpayer contributions correlate with higher profits or reduced losses?
Cost Efficiency

Calculate the cost per athlete or cost per event for each Olympiad. This can provide insight into how efficiently the games were managed financially.
Return on Investment (ROI)

Evaluate the ROI for each Olympics by comparing the total costs to the profit/loss. Highlight which Olympiads had the best and worst ROI.
Seasonal Comparison

Compare the financial dynamics between Summer and Winter Olympics. Which season tends to be more expensive or more profitable?
Geopolitical Influence

Investigate if certain regions (e.g., North America, Europe, Asia) have differing financial dynamics when hosting the Olympics. Are some regions more likely to see higher taxpayer contributions or profits?
Inflation Adjustment

Adjust the costs for inflation to compare the real value of hosting costs across different years. This can provide a more accurate comparison of financial data over time.
